In [9]:
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import math

# Adjust the path for c2qa and bosonic-qiskit modules
module_path = os.path.abspath(os.path.join('..', '..', '..'))  # Moves three directories up
if module_path not in sys.path:
    sys.path.append(module_path)

# Add the `playground` directory to the Python path
playground_path = Path.cwd().parent.parent  # Move two levels up to `playground`
if str(playground_path) not in sys.path:
    sys.path.append(str(playground_path))


# Qiskit and related imports
import qiskit
from qiskit.quantum_info import state_fidelity, Statevector, SparsePauliOp, Operator, Clifford
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate, UnitaryGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter
import qutip as qt
from qiskit import transpile


# Project-specific imports
import c2qa
import c2qa.util as util

[ref](https://arxiv.org/pdf/2312.05344)


Here we focus on the axial-vector term of the pion-nucleon interaction Hamiltonian. We consider a three dimensional lattice $N\times N\times N$, with spacing $a_L$. The pion field at lattice site $x$ is represented by $\pi_I(x)$ for the isospin indices $I\in\{1,2,3\}$. The fermionic annihilation and creation operators at site $x$ for species $\sigma$ are  $a_\sigma(x)$ and $a_\sigma^\dagger(x)$, respectively, where $\sigma$ runs over protons, neutrons, and their spin states: $\sigma \in \{\uparrow, \downarrow\} \times \{\text{proton}, \text{neutron}\}$, i.e. $ a_{00}(x) = a_{\uparrow p}(x)$,  $ a_{01}(x) = a_{\uparrow n}(x)$,  $ a_{10}(x) = a_{\downarrow p}(x)$,  $ a_{11}(x) = a_{\downarrow n}(x)$. The axial-vector term is
$$
H_{AV} = \frac{g_A}{2 f_\pi} \sum_{x} \sum_{\alpha,\beta,\gamma,\delta} \sum_{I=1}^3 \sum_{S=1}^3 a^\dagger_{\alpha\beta}(x) \left[\tau_I\right]_{\beta\delta} \left[\sigma_S\right]_{\alpha\gamma} \partial_S\,\pi_I(x) \; a_{\gamma\delta}(x),
$$
where we define the Pauli matrices $\sigma_S$ (with $S\in\{1,2,3\}$) as operators on the spin space, and the matrices $\tau_I$ (with $I\in\{1,2,3\}$) as operators on the isospin space, with

$$
\sigma_1 = \tau_1 = X,\quad \sigma_2 = \tau_2 = Y,\quad \sigma_3 = \tau_3 = Z.
$$

Here, the notation $\left[\sigma_S\right]_{\alpha\beta}$ indicates the $(\alpha,\beta)$ entry of the matrix $\sigma_S$. From which, we get 36 non-zero terms.

We can quantize the pion fields in the standard way such that we obtain an expansion in terms of the (bosonic) creation $b_k^\dag$ and anhillation $b_k$ operators,
$$
\pi_I(x) = \frac{1}{\sqrt{V}} \sum_{k} \frac{1}{\sqrt{2\omega_k}} \left( a_I(k) e^{\frac{i2\pi\, k\cdot x}{N}} + a_I^\dagger(k) e^{\frac{-i2\pi\, k\cdot x}{N}} \right),
$$
with the momentum vector is defined as $k = (k_1,k_2,k_3), k_i \in \left[-\frac{N}{2},\,\frac{N}{2}-1\right],$ i.e. $k$ labels the corresponding momentum mode, and $V = N^3$.

### Highest Weight
The highest weight term in the sum will be e.g.
$$
\frac{\pi_1(x + a_L \hat{n}_1) - \pi_1(x)}{a_L}
\left[
a_{\uparrow p}^\dagger(x) a_{\downarrow n}(x)
\right],
$$
where $\hat{n}_j$ is the unit vector along the cartesian coordinate $j$.

### Lowest Weight 
The lowest weight term in the sum will be e.g.
$$
\frac{\pi_3(x + a_L \hat{n}_3) - \pi_3(x)}{a_L}
\left[
a_{\uparrow p}^\dagger(x) a_{\uparrow p}(x)
\right],
$$
where $\hat{n}_j$ is the unit vector along the cartesian coordinate $j$.

# 2 Site Model
In our two-site model, we have only one spatial link, so a full three-dimensional derivative is not defined. Therefore, we restrict the derivative to a single direction. For concreteness, we choose $S=3$,
$$
H_{AV}^{\text{2-site}} = \frac{g_A}{2 f_\pi} \sum_{x=1}^2 \sum_{\alpha,\beta,\gamma,\delta} \sum_{I=1}^{3} a^\dagger_{\alpha\beta}(x) \left[\tau_I\right]_{\beta\delta} \left[\sigma_3\right]_{\alpha\gamma} \partial_3\,\pi_I(x) \; a_{\gamma\delta}(x).
$$

In this reduced model, the only nontrivial derivative is along the third spatial direction. The only non-zero terms will occur when $\alpha = \gamma$, hence we can write 
$$
H_{AV}^{\text{2-site}} = \frac{g_A}{2 f_\pi}\, \sum_{x=1}^{2} \sum_{\beta, \delta} \sum_{I=1}^{3} \left[ a_{0\beta}^\dagger(x) [\tau_I]_{\beta\delta} \, \partial_3 \pi_I(x) \, a_{0\delta}(x) - a_{1\beta}^\dagger(x) [\tau_I]_{\beta\delta} \, \partial_3 \pi_I(x) \, a_{1\delta}(x) \right],
$$
which we can see will give us a total of 12 non-zero terms per lattice site in the sum.

### A Single Pion Isospin
To begin, we choose a simplification of $I = 3$. Thus, our Hamiltonian is
$$
H_{AV}^{\text{2-site}}\Big|_{I=3} = \frac{g_A}{2 f_\pi}\sum_{x=1}^{2}\left\{
a^\dagger_{00}(x)a_{00}(x) - a^\dagger_{01}(x)a_{01}(x) - a^\dagger_{10}(x)a_{10}(x) + a^\dagger_{11}(x)a_{11}(x)
\right\}\partial_3\pi_3(x)
$$
$$
= \frac{g_A}{2 f_\pi}\sum_{x=1}^{2}\left\{
a^\dagger_{00}(x)a_{00}(x) - a^\dagger_{01}(x)a_{01}(x) - a^\dagger_{10}(x)a_{10}(x) + a^\dagger_{11}(x)a_{11}(x)
\right\} \frac{\pi_3(x + a_L)-\pi_3(x)}{a_L}.
$$

We can now map this expression to spin-boson degrees of freedom. For each fermionic mode at lattice site $x$, we use the mapping
$$
a_{ij}^\dagger(x)a_{ij}(x) = \frac{1}{2}\Bigl( I + Z_{ij}(x) \Bigr), \quad ij\in\{00, 01, 10, 11\},
$$
which we can see means the identity pieces cancel (since $I - I - I + I = 0$), leaving only a combination of the $Z$ operators. Thus,
$$
\begin{aligned}
H_{AV}^{\text{2-site}}\Big|_{I=3} &= \frac{g_A}{4 f_\pi\, a_L\sqrt{2}} \sum_{x=1}^{2} \Bigl[ Z_{00}(x)-Z_{01}(x)-Z_{10}(x)+Z_{11}(x)\Bigr] \\
&\quad \times \sum_{m=1}^{2} \frac{1}{\sqrt{2\omega_m}} \Bigl\{ b_m\,\Bigl(e^{i\pi m (x+1)}-e^{i\pi m x}\Bigr) + b_m^\dagger\,\Bigl(e^{-i\pi m (x+1)}-e^{-i\pi m x}\Bigr) \Bigr\}\,.
\end{aligned}
$$
Given that for any even momentum index the phase factors cancel in the finite‐difference derivative, the final Hamiltonian is

$$
\boxed{
H_{AV}^{\text{2-site}}\Big|_{I=3} = \frac{g_A}{4f_\pi\,a_L\sqrt{\omega_1}}
\left\{
\Bigl[Z_{00}(1)-Z_{01}(1)-Z_{10}(1)+Z_{11}(1)\Bigr]
-\Bigl[Z_{00}(2)-Z_{01}(2)-Z_{10}(2)+Z_{11}(2)\Bigr]
\right\}(b_1+b_1^\dagger).
}
$$
This corresponds to 8 conditonal displacements.

In [10]:
def create_quantum_circuit(num_sites=2, num_qubits_per_qumode=1):
    """Create and initialize the circuit
    We have 1 pion isospin, so 1 oscillator per site. 
    We have 4 fermionic species, so 4 qubits per site.
    """
    num_qubits = 4*num_sites
    # Create registers
    qmr = c2qa.QumodeRegister(num_qumodes=num_sites, 
                             num_qubits_per_qumode=num_qubits_per_qumode)
    qbr = qiskit.QuantumRegister(num_qubits)
    circuit = c2qa.CVCircuit(qmr, qbr)

    return circuit, qmr, qbr

def trotter_two_site_H_av(circuit, qmr, qbr, prefactor, dt, steps):
    for step in range(steps):
        # --- Site 1 ---
        # qubit 0 (Z_00): +1 coefficient
        circuit.cv_c_d(-1j * prefactor * dt, qmr[0], qbr[0])
        # qubit 1 (Z_01): -1 coefficient
        circuit.cv_c_d( 1j * prefactor * dt, qmr[0], qbr[1])
        # qubit 2 (Z_10): -1 coefficient
        circuit.cv_c_d( 1j * prefactor * dt, qmr[0], qbr[2])
        # qubit 3 (Z_11): +1 coefficient
        circuit.cv_c_d(-1j * prefactor * dt, qmr[0], qbr[3])
        
        # --- Site 2 ---
        # overall sign flips, so effective factor = -prefactor.
        # qubit 4 (Z_00): +1 
        circuit.cv_c_d(-1j * (-prefactor) * dt, qmr[1], qbr[4])
        # qubit 5 (Z_01): -1 
        circuit.cv_c_d( 1j * (-prefactor) * dt, qmr[1], qbr[5])
        # qubit 6 (Z_10): -1 
        circuit.cv_c_d( 1j * (-prefactor) * dt, qmr[1], qbr[6])
        # qubit 7 (Z_11): +1 
        circuit.cv_c_d(-1j * (-prefactor) * dt, qmr[1], qbr[7])
    
    return circuit

def run_simulation(circuit):
    state, _, _ = c2qa.util.simulate(circuit)
    return state

def simulate_two_site_H_av(num_sites, num_qubits_per_qumode, prefactor, dt, steps):
    """
    Simulate the two-site axial vector term of the pion-nucleon interaction Hamiltonian
    using a trotter decomp for a single pion isospin.
    """
    num_sites = 2
    
    # Create and initialize the circuit
    circuit, qmr, qbr = create_quantum_circuit(num_sites, num_qubits_per_qumode)
    
    # Apply Trotter steps
    circuit = trotter_two_site_H_av(circuit, qmr, qbr, prefactor, dt, steps)
    
    # Run the simulation
    state = run_simulation(circuit)
    
    return state


###  Loschmidt echo